# Call model without context

In [1]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

callbacks = [StreamingStdOutCallbackHandler()]

In [2]:
from langchain.llms import Ollama

model = "mistral"

llm = Ollama(model=model, callbacks=callbacks)

In [3]:
query = "What is ornl-next?"

llm.invoke(query)

 ORNL-Next is a collaborative research and development effort between Oak Ridge National Laboratory (ORNL) and several industry partners to design, build, and demonstrate a versatile and scalable advanced manufacturing technology system. The main goal of ORNL-Next is to accelerate the transition of novel materials and technologies from laboratory scale to industrial production.

The ORNL-Next project focuses on developing an integrated, high-throughput manufacturing platform that can produce complex parts with various materials using additive manufacturing (AM), also known as 3D printing, and other advanced manufacturing techniques. The system is designed to be adaptable, allowing for the rapid integration of new tools, processes, and materials as they are developed.

The project is significant because it aims to bridge the gap between laboratory research and industrial-scale production, making it easier for innovations in materials science, engineering, and manufacturing to reach comm

' ORNL-Next is a collaborative research and development effort between Oak Ridge National Laboratory (ORNL) and several industry partners to design, build, and demonstrate a versatile and scalable advanced manufacturing technology system. The main goal of ORNL-Next is to accelerate the transition of novel materials and technologies from laboratory scale to industrial production.\n\nThe ORNL-Next project focuses on developing an integrated, high-throughput manufacturing platform that can produce complex parts with various materials using additive manufacturing (AM), also known as 3D printing, and other advanced manufacturing techniques. The system is designed to be adaptable, allowing for the rapid integration of new tools, processes, and materials as they are developed.\n\nThe project is significant because it aims to bridge the gap between laboratory research and industrial-scale production, making it easier for innovations in materials science, engineering, and manufacturing to reach

Without providing any contex, LLM model is using the keywords "ORNL" to make up most of the context.

## Setup context

Let's see what will happen if we provide a context to the model.

In [4]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("../data/mantid-fork.pdf")
pages = loader.load_and_split()

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(pages)

In [6]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [7]:
from langchain.vectorstores import Chroma

target_source_chunks = 4

db = Chroma.from_documents(all_splits, embeddings)
retriever = db.as_retriever(search_kwargs={"k": target_source_chunks})

In [8]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
)

In [9]:
print(query)

What is ornl-next?


In [10]:
rst = qa.invoke(query)

 Ornl-next is a branch in the Git repository where new developments and changes are automatically built and published to the mantid-ornl channel on a nightly basis. It serves as a staging area for new features before they are merged into the stable ornal and QA branches.

In [11]:
print(rst)

{'query': 'What is ornl-next?', 'result': ' Ornl-next is a branch in the Git repository where new developments and changes are automatically built and published to the mantid-ornl channel on a nightly basis. It serves as a staging area for new features before they are merged into the stable ornal and QA branches.'}
